In [1]:
import json
import os

import numpy as np
import numpy as np
import pandas as pd

In [2]:
def get_simulation_metafile_path(img_id, eps_am, resolution, Wa_n=1e-3, Wa_p=1, kr=1, gamma=15, refined=False):
    if refined:
        return os.path.join("output/reaction_distribution/470-470-45/15-30/", str(int(img_id)), str(eps_am), str(resolution), f"{Wa_n}-{Wa_p}-{kr}", str(gamma), "simulation.json")
    else:
        return os.path.join("output/reaction_distribution/470-470-45/15-30/", str(int(img_id)), str(eps_am), "unrefined", str(resolution), f"{Wa_n}-{Wa_p}-{kr}", str(gamma), "simulation.json")

In [3]:
def read_simulation_metafile(metafile_path):
    with open(metafile_path, "r") as fp:
        return json.load(fp)

In [6]:
df_rows = []
vol_fracs = np.linspace(0.1, 0.6, num=6, dtype=np.float16)
image_ids = [1, 11, 16, 22]
wagner_nums = [0.1, 1, 10]
krs = [0.01, 1, 100]

for eps_am in vol_fracs:
    for Wa_p in wagner_nums:
        for img_id in image_ids:
            for kr in krs:
                sim_metafile = get_simulation_metafile_path(img_id, eps_am, 5.0, kr)
                try:
                    row_data = read_simulation_metafile(sim_metafile)
                except FileNotFoundError:
                    continue
                row_data["eps_am"] = eps_am
                df_rows.append(row_data)

data = pd.DataFrame(df_rows, dtype=np.float)

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

### $\frac{\kappa}{\sigma}$ = 0.01

In [ ]:
df_001 = df[np.isclose(df["kr"], 0.01)]
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_001['eps_am'], df_001['Total resistance [Ω.cm2]'])
ax.set_box_aspect(1)
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\varepsilon_{\mathrm{+ve\ AM}}$', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()

### $\frac{\kappa}{\sigma}$ = 1

In [ ]:
df_1 = df[np.isclose(df["kr"], 1)]
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_1['eps_am'], df_1['Total resistance [Ω.cm2]'])
ax.set_box_aspect(1)
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\varepsilon_{\mathrm{+ve\ AM}}$', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()

### $\frac{\kappa}{\sigma}$ = 100

In [5]:
df_100 = df[np.isclose(df["kr"], 100)]
fig, ax = plt.subplots(figsize=(5, 4.5))
ax.plot(df_100['eps_am'], df_100['Total resistance [Ω.cm2]'])
ax.set_box_aspect(1)
ax.set_ylabel(r'R [$\Omega\mathrm{cm}^2$]', fontdict={'fontsize': 'xx-large'})
ax.set_xlabel(r'$\varepsilon_{\mathrm{+ve\ AM}}$', fontdict={'fontsize': 'xx-large'})
plt.tight_layout()

NameError: name 'df' is not defined